In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
sys.path.append('/home/demitau/ownCloud/Current/NeuroSci/MSCA_sci/code/data_proc')
data_dir = '/home/demitau/data_duss'
subjects_dir = '/home/demitau/data_duss/MRI/subjects'

import mne
# import utils  #my code
# import json
import matplotlib.pyplot as plt
import numpy as np
# import utils_preproc as upre
# import globvars as gv

rawnames = ['S01_off_hold', 'S01_off_move']

In [ ]:
from os.path import join as pjoin
raws = []
for rn in rawnames:
    fnf = pjoin(data_dir,rn + '_hires-raw.fif')
    raw = mne.io.read_raw_fif(fnf)
    
    fnf = pjoin(data_dir,rn+'_anns.txt')
    anns = mne.read_annotations(fnf)
    raw.set_annotations(anns)
    raws += [raw]

In [ ]:
mne.compute_covariance?

In [ ]:
# todo: make epochs

In [ ]:
!OSCBAGDIS_DATAPROC_CODE=/home/current_ownCloud/NeuroSci/code/data_proc
!OSCBAGDIS_DATA=/home/demitau/data_duss

In [ ]:
!echo  "$OCBAGDIS_DATAPROC_CODE"

In [ ]:
os.environ['OCBAGDIS_DATAPROC_CODE'] = '/home/current_ownCloud/NeuroSci/code/data_proc'

In [ ]:
os.path.expandvars('$OCBAGDIS_DATAPROC_CODE')

In [ ]:
%autoreload

In [ ]:
from globvars import gp

In [ ]:
event_id = {}

In [ ]:
#def ann2flevents(anns):
rawname = rawnames[1]
anns = mne.read_annotations(pjoin(data_dir,f'{rawname}_anns.txt' ))
event_id = {}
for ann in anns:
    if ann['description'] in gp.int_types_basic_sided:
        ind = gp.int_types_basic_sided.index(ann['description'])
        event_id[ann['description']] = ind
    print(ann)

In [ ]:
n_cycles = 7
wnddur = 1 # in sec
min_freq = 3
fleid = 1000
edge_artif_len = (1./min_freq) * (n_cycles/2.)
events = mne.event.make_fixed_length_events(raw, id=fleid, start=edge_artif_len, 
    duration=wnddur + 2*edge_artif_len, overlap = edge_artif_len * 2)
event_id['wnd_start'] = fleid

In [ ]:
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'],
                          event_id=event_id);

In [ ]:
picks = mne.pick_types(raw.info, meg=True, eeg=False, eog=True,
                       stim=False, exclude='bads')
baseline = None
reject = None

In [ ]:
tmin = 0-edge_artif_len
tmax = wnddur + edge_artif_len
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                    picks=picks, baseline=baseline,
                    reject=reject)

In [ ]:
raw.load_data()

In [ ]:
# There is a function to create EOG epochs:
eog_epochs = mne.preprocessing.create_eog_epochs(raw.copy().filter(1, None))
eog_epochs.average().plot_joint();

In [ ]:
# same business, same issue for ECG
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw.copy().filter(1, None))
ecg_epochs.average().plot_joint()

In [ ]:
import mne
data_cov = mne.compute_covariance(epochs, method='empirical', rank='info')
mne.viz.plot_cov(data_cov, info=epochs.info)

In [ ]:
# find artif in raw using ICA fitted to raw, annotate
# lowpass and compute eog and ecg, remove from original using same demixing matrix
# notch
# max filter?

In [ ]:
from mne.preprocessing import annotate_muscle_zscore

In [ ]:
raw.load_data()

In [ ]:
annotate_muscle_zscore?

In [ ]:
threshold_muscle = 5  # z-score
# Choose one channel type, if there are axial gradiometers and magnetometers,
# select magnetometers as they are more sensitive to muscle activity.
annot_muscle, scores_muscle = annotate_muscle_zscore(
    raw, ch_type="mag", threshold=threshold_muscle, min_length_good=0.2,
    filter_freq=[110, 140])

In [ ]:
fig, ax = plt.subplots()
ax.plot(raw.times, scores_muscle)
ax.axhline(y=threshold_muscle, color='r')
ax.set(xlabel='time, (s)', ylabel='zscore', title='Muscle activity')

In [ ]:
raw.annotations


In [ ]:
order = np.arange(144, 164)
raw.set_annotations(annot_muscle)
raw.plot(start=5, duration=20, proj=0) #, order=order)

# CB

In [ ]:
import mne
from mne import setup_source_space, setup_volume_source_space
from mne.datasets import sample

print(__doc__)

data_path = sample.data_path()
subjects_dir = data_path / 'subjects'
subject = 'sample'
aseg_fname = subjects_dir / 'sample' / 'mri' / 'aseg.mgz'

# %%
# Setup the source spaces

# setup a cortical surface source space and extract left hemisphere
surf = setup_source_space(subject, subjects_dir=subjects_dir, add_dist=False)
lh_surf = surf[0]

# setup a volume source space of the left cerebellum cortex
volume_label = 'Left-Cerebellum-Cortex'
sphere = (0, 0, 0, 0.12)
lh_cereb = setup_volume_source_space(
    subject, mri=aseg_fname, sphere=sphere, volume_label=volume_label,
    subjects_dir=subjects_dir, sphere_units='m')

# Combine the source spaces
src = surf + lh_cereb

In [ ]:
setup_source_space?